In [1]:
import numpy as np
import h5py as h5
from pyproj import Proj,transform
from osgeo import gdal
from math import floor
import numpy as np
import osr
from glob import glob

In [2]:
## Reading GEDI & Sentinel2

class readGEDI():
    def __init__(self,fileList):
        gedi_f = h5.File('/home/s1949330/Documents/initial_data/GEDI02_B_2020_04_30_2020124194427_O07882_01_T02809_02_003_01_V002.h5', 'r')
        beamList = ['BEAM0000','BEAM0001','BEAM0010','BEAM0011','BEAM0101','BEAM0110','BEAM1000','BEAM1011']           
        self.x = np.empty((), dtype = float)
        self.y = np.empty((), dtype = float)
        self.sens = np.empty((), dtype = float)
        self.cover = np.empty((), dtype = float)
        for beam in beamList:
            self.x = np.append(self.x, np.array(gedi_f[beam]['geolocation/lon_lowestmode']))
            self.y = np.append(self.y, np.array(gedi_f[beam]['geolocation/lat_lowestmode']))
            self.sens = np.append(self.sens, np.array(gedi_f[beam]['sensitivity']))
            self.cover = np.append(self.cover, np.array(gedi_f[beam]['cover']))   
        return
            
    def reprojectData (self,outEPSG):
        '''Rerpoject footprint coordinates'''
        inProj = Proj("epsg:4326")
        outProj = Proj("epsg:"+str(outEPSG))
        x,y=transform(inProj,outProj,gedi.x,gedi.y)
        gedi.x=x
        gedi.y=y
        return  
    

class readSentinel():
    def __init__(self,fileName):
        ds = gdal.Open(fileName)
        proj = osr.SpatialReference(wkt = ds.GetProjection())
        self.epsg = int(proj.GetAttrValue('AUTHORITY', 1))
        self.nX = ds.RasterXSize
        self.nY = ds.RasterYSize
        transform_ds = ds.GetGeoTransform()
        self.xOrigin = transform_ds[0]
        self.yOrigin = transform_ds[3]
        self.pixelWidth = transform_ds[1]
        self.pixelHeight = transform_ds[5]
        self.data = ds.GetRasterBand(1).ReadAsArray(0, 0, self.nX, self.nY)
        return
    
    def coarsen(self,res):
        '''Coarsen resolution of data layer'''    
        # dimensions of output image
        newX = floor(self.nX * self.pixelWidth / res)
        newY = floor(self.nY * self.pixelHeight / (-1 * res))
        # allocate space
        newData = np.zeros((newX,newY), dtype = float)
        contN = np.zeros((newX,newY), dtype = int)
        # loop iover inpuit image and take rolling sum
        for i in range(0,self.nX):
            print('Column', i , 'of', self.nX)
            for j in range(0, self.nY):
                xInd = floor(i * self.pixelWidth / res)
                yInd = floor(j * -1 * self.pixelHeight / res)
                print(i,j,xInd,yInd,res,self.pixelWidth,self.pixelHeight)
                newData[xInd,yInd] += self.data[i,j]
                contN[xInd,yInd] += 1
                
        # normalise means
        newData[contN > 0] = newData[contN > 0] / contN[contN > 0]
        newData[contN == 0] = -999
        self.pixelWidth = res
        self.pixelHeight = -1 * res
        self.data = newData
        return


[-0.1        -0.17432582 -0.17403249 ... 43.89469689 43.89499315
 43.8952898 ]
[  0.         -43.36818741 -43.36804818 ...  -0.07988222  -0.07946151
  -0.07904034]
[0.         4.61821699 4.1790204  ... 0.76948482 0.78958929 0.72204185]
[ 2.00000000e+00 -9.99900000e+03 -9.99900000e+03 ...  1.97091792e-02
  5.04121557e-03  1.79270748e-02]


In [3]:
### Reading Sentinel-2


    
    


In [14]:
# Run Sentinel-2 Coarsening

dir='/home/s1949330/Documents/initial_data'
sentList=glob(dir+'/*.jp2')
print(sentList)

s2 = np.empty((12),dtype = readSentinel)
useList = np.full((12),True,dtype = bool)
print(useList)
        
i = 0
for filename in sentList:
    s2[i] = readSentinel(filename)  
    if s2[i].pixelWidth == 60:
        useList[i] = False
    elif s2[i].pixelWidth == 10:
        useList[i] = False
        #s2[i].coarsen(20)
    #i = i + 1 
    

['/home/s1949330/Documents/initial_data/T36LWK_A025400_20200503T075551_B07.jp2', '/home/s1949330/Documents/initial_data/T36LWK_A025400_20200503T075551_B08.jp2', '/home/s1949330/Documents/initial_data/T36LWK_A025400_20200503T075551_B05.jp2', '/home/s1949330/Documents/initial_data/T36LWK_A025400_20200503T075551_B8A.jp2', '/home/s1949330/Documents/initial_data/T36LWK_A025400_20200503T075551_B01.jp2', '/home/s1949330/Documents/initial_data/T36LWK_A025400_20200503T075551_B04.jp2', '/home/s1949330/Documents/initial_data/T36LWK_A025400_20200503T075551_B06.jp2', '/home/s1949330/Documents/initial_data/T36LWK_A025400_20200503T075551_B12.jp2', '/home/s1949330/Documents/initial_data/T36LWK_A025400_20200503T075551_B02.jp2', '/home/s1949330/Documents/initial_data/T36LWK_A025400_20200503T075551_B11.jp2', '/home/s1949330/Documents/initial_data/T36LWK_A025400_20200503T075551_B09.jp2', '/home/s1949330/Documents/initial_data/T36LWK_A025400_20200503T075551_B03.jp2']
[ True  True  True  True  True  True  T

In [15]:
# GEDI array intersect with Sentinel-2 coordinates

nGedi = gedi.cover.shape[0]
nSent = s2.shape[0]
s2out = np.full((nGedi,nSent), -999, dtype = float)
print(s2out.shape)

i = 0
for i in range(0,nSent):
    xInd = np.floor((gedi.x - s2[i].xOrigin) / s2[i].pixelWidth)
    yInd = np.floor((gedi.y - s2[i].yOrigin) / s2[i].pixelHeight)
    #s2ref[:i] = s2[i].data(xInd,yInd)

      
# Reproject GEDI coordinates into Sentinel 2 projection
gedi.reprojectData(s2[0].epsg)
print(gedi.x.shape, gedi.y.shape)
print(gedi.x, gedi.y)


# Align Sentinel-2 and GEDI
useMask = [(xInd >= 0), (xInd < readSentinel.nX), (yInd >= 0), (yInd < readSentinel.nY)]
aligned = s2[i].data[xInd[useMask], yInd[useMask]]
print(aligned)


(914390, 12)


AttributeError: type object 'readSentinel' has no attribute 'data'

In [ ]:
# Extract S2 data into GEDI structure

In [ ]:
# write all data to CSV


# end notebook. Make new notebook